## Introduction:

Main Goals:

1. Identify the recipients that will engage with the campaign.
2. Maximise the campaign’s revenue.


Comments

- The dataset contains only 5% of donors.
- The donations are usually smaller than $20.
- This data is quite noisy, high dimensional.
- There is an inverse relationship between the probability to donate and the amount donated.


Link for dataset and some analysis ==> 

https://github.com/rebordao/kdd98cup

https://github.com/bobbyantonio/KDD98/blob/master/CleanData.py

- Github solutions ==>
https://github.com/rebordao/kdd98cup


- Siraj notebook for a better data visualization:

https://www.youtube.com/watch?v=yQsOFWqpjkE

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.python.ops import rnn, rnn_cell
from sklearn.model_selection import train_test_split
import pandas as pd
import random

import csv
import sys

# import datetime as dt

## warnings
import warnings
warnings.filterwarnings("ignore")

## Loading the dataset:

In [2]:
def load_data():
    return pd.read_csv('tuple.csv', header = 0)

In [3]:
## I have to run it over all the data not just 100 records
df = load_data()
df.head(3)

,r0,f0,m0,ir0,if0,a,r1,f1,m1,ir1,if1,rew
0,0,0,0,0,0,5,0,1,9,0,1,9
1,0,0,0,0,0,5,1,0,0,0,1,0
2,0,0,0,0,0,5,0,1,6,0,1,6


In [4]:
def df_split():
    
    df = load_data()
    train, test = train_test_split(df, test_size = 0.999)  # split data to 50-50 cross validate, Roger 1.6*1000,000
    train = train.convert_objects(convert_numeric=True)
    
    train_y = train['rew']
    train_x = train[train.columns.difference(['rew'])]
    
    return train_x, train_y

In [5]:
def tuple_():
        
    train_x, train_y = df_split()
    
    next_actions = np.zeros([len(train_x), 12])
    
    for i in xrange(np.shape(train_x)[1]):
        next_actions[:, i] = i
    
    
    # next_state_next_action
    tuplesMx0 = np.column_stack((train_x, next_actions[:,0]))
    tuplesMx1 = np.column_stack((train_x, next_actions[:,1]))
    tuplesMx2 = np.column_stack((train_x, next_actions[:,2]))
    tuplesMx3 = np.column_stack((train_x, next_actions[:,3]))
    tuplesMx4 = np.column_stack((train_x, next_actions[:,4]))
    tuplesMx5 = np.column_stack((train_x, next_actions[:,5]))
    tuplesMx6 = np.column_stack((train_x, next_actions[:,6]))
    tuplesMx7 = np.column_stack((train_x, next_actions[:,7]))
    tuplesMx8 = np.column_stack((train_x, next_actions[:,8]))
    tuplesMx9 = np.column_stack((train_x, next_actions[:,9]))
    tuplesMx10 = np.column_stack((train_x, next_actions[:,10]))
    tuplesMx11 = np.column_stack((train_x, next_actions[:,11]))
    
    return tuplesMx0, tuplesMx1, tuplesMx2, tuplesMx3, tuplesMx4, tuplesMx5, tuplesMx6, tuplesMx7, tuplesMx8, tuplesMx9, tuplesMx10, tuplesMx11

In [6]:
# tuplesMx0, tuplesMx1, tuplesMx2, tuplesMx3, tuplesMx4, tuplesMx5, tuplesMx6, tuplesMx7, tuplesMx8, tuplesMx9, tuplesMx10, tuplesMx11  = tuple_()
# tuplesMx0, tuplesMx1, tuplesMx2, tuplesMx3, tuplesMx4, tuplesMx5, tuplesMx6, tuplesMx7, tuplesMx8, tuplesMx9, tuplesMx10, tuplesMx11

## Regression phase:

Before performing the prediction task .. let's split the data to training and validation sets .. 

To avoid any problems in prediction by having string variables .. let's binarize (catergorize) all the variables .. 

Guidance ==> https://pythonprogramming.net/rnn-tensorflow-python-machine-learning-tutorial/

In [13]:
gamma = 0.9
Num_itrs = 2  # no loop 3leha ( w dah el sa7) .. we just have to run the whole algo. 10 times and report the avg. results

num_epoch = 1 #23 #epochs are cycles of Feedforward and Backprob
## el mafrood yeb2a feh loop 3la el epochs elli heyya el steps .. w avg. reward per step is calculated 
batch_size = 5
chunkSize = 1

train_x, train_y = df_split()

n_nodes_hl1 = 40 #np.shape(train)[1]
n_nodes_hl2 = 15 #np.shape(train)[0]
# NUM_STATES = np.shape(train)[1]
NUM_DIM =  np.shape(train_x)[1]
num_nodes = np.shape(train_x)[0]

num_unrollings = 5

# best_actions = np.zeros([np.shape(train)[0], batch_size])
Q_optimal = [] #np.zeros([np.shape(curr_state_current_action)[0], len(df['ACCOUNT_STATUS'].unique())])

########################################################################################################

## for training 
x = tf.placeholder(tf.float32, shape=[num_nodes, NUM_DIM])
y =  tf.placeholder(tf.float32, shape=[num_nodes])

# ## for testing
# x_ = tf.placeholder(tf.float32, shape=[NUM_DIM, None])
# y_ =  tf.placeholder(tf.float32, shape=[NUM_DIM, None])

########################################################################################################

# Input gate: input, previous output, and bias.
ix = tf.Variable(tf.random_uniform([NUM_DIM, num_nodes],0,1, dtype = tf.float32, seed = 0))# init_weights_RNN([n_nodes_hl1, NUM_ACTIONS])
im = tf.Variable(tf.random_uniform([num_nodes, 1],0,1, dtype = tf.float32, seed = 0))
ib = tf.Variable(tf.zeros([1, num_nodes], dtype = tf.float32))

# Forget gate: input, previous output, and bias.
fx = tf.Variable(tf.random_uniform([NUM_DIM, num_nodes],0,1, dtype = tf.float32, seed = 0))
fm = tf.Variable(tf.random_uniform([num_nodes, num_nodes],0,1, dtype = tf.float32, seed = 0))
fb = tf.Variable(tf.zeros([1, num_nodes], dtype = tf.float32))

# Memory cell: input, state and bias.                             
cx = tf.Variable(tf.random_uniform([NUM_DIM, num_nodes],0,1, dtype = tf.float32, seed = 0))
cm = tf.Variable(tf.random_uniform([num_nodes, num_nodes],0,1, dtype = tf.float32, seed = 0))
cb = tf.Variable(tf.zeros([1, num_nodes], dtype = tf.float32))

# Output gate: input, previous output, and bias.
ox = tf.Variable(tf.random_uniform([NUM_DIM, num_nodes],0,1, dtype = tf.float32, seed = 0))
om = tf.Variable(tf.random_uniform([num_nodes, num_nodes],0,1, dtype = tf.float32, seed = 0))
ob = tf.Variable(tf.zeros([1], dtype = tf.float32))

# Variables saving state across unrollings.
saved_output = tf.Variable(tf.zeros([num_nodes, num_nodes], dtype = tf.float32), trainable = False) #reversed

saved_state = tf.Variable(tf.zeros([num_nodes, num_nodes], dtype = tf.float32), trainable = False) #reversed

# Classifier weights and biases.
w = tf.Variable(tf.random_uniform([num_nodes, 1], 0, 1, dtype = tf.float32, seed = 0))
b = tf.Variable(tf.zeros([1], dtype = tf.float32))

In [14]:
# Definition of the cell computation.
# this method takes single cell and returns single number 
def lstm_cell(i, o, state):
    input_gate = tf.sigmoid(tf.matmul(i, ix))# + tf.matmul(o, im) + ib)
    forget_gate = tf.sigmoid(tf.matmul(i, fx) + tf.matmul(o, fm) + fb)
    update = tf.matmul(i, cx) + tf.matmul(o, cm) + cb
    state = forget_gate * state + input_gate * tf.tanh(update)
    output_gate = tf.sigmoid(tf.matmul(i, ox) + tf.matmul(o, om) + ob)
    return output_gate*tf.tanh(state), state

In [20]:
def lstm_model(train_x, saved_output, saved_state):
    # Unrolled LSTM loop.    
#     output_ = list()
    output = saved_output  # row !
    state = saved_state  # row !

    
    output_, state = lstm_cell(train_x.values.astype(np.float32), tf.cast(output, tf.float32), tf.cast(state, tf.float32)) 
    
    # astype('U') .. to convert numpy array to string ..
#     for i in xrange(np.shape(train)[0]):## el loop faydetha to copy the next line that is just for single unit 
#         output_, state = lstm_cell(train.values[i, None].astype(np.float32), tf.cast(output, tf.float32), tf.cast(state, tf.float32)) 

    
        # in case the last values are saved !
    with tf.control_dependencies([saved_output.assign(output), saved_state.assign(state)]):
        model_output = tf.matmul(output_, w) + b # outputs single value
    
    
    return model_output  ## the output for the whole model

In [21]:
def train_lstm_model():

    loss_RNN = []
    
    model_output = lstm_model(train_x, saved_output, saved_state) #output here is a vector

    print model_output
    cost = tf.reduce_mean(tf.square(y - model_output))

    optimize = tf.train.RMSPropOptimizer(0.0001, 0.99).minimize(cost) 

    sess = tf.Session()
    init = tf.global_variables_initializer()  ## updated version from initialize_all_variables :) 
    sess.run(init)

    
    Predicted_reward = sess.run(model_output, feed_dict={x:train_x.values.astype(np.float32), y:train_y.values.astype(np.float32)})
    
    
#     # Cost calculation
#     for step in xrange(1000):
#         l,_ = sess.run([cost, optimize], feed_dict={x:train, y: train.values})
    
#         if step % 100 == 0:
#             loss_RNN.append(l)

# #     Y_pred = tf.zeros([len(y_test),1])

# #     # predict 
#     test_pred2 = sess.run(Y_pred, feed_dict={x_: test.values})

# #    ## rms to test ..
# #     cost_test = tf.reduce_mean(tf.square(test.values.astype(np.float32) - test_pred2))
    
    
# #     rmse_val = sess.run(cost_test, feed_dict={x_:test.values.astype(np.float32), y_: test_pred2})
     
    sess.close()
    
#     for i in xrange(np.shape(Predicted_reward)[0]):
#         if (Predicted_reward[i] <= 0):
#             print i
#             Predicted_reward[i] = 0
#         else:
#             Predicted_reward[i] 
            
    
    return Predicted_reward#.astype(int) #saved_state, saved_output, cost_values

In [22]:
train_lstm_model()

Tensor("add_39:0", shape=(2099, 1), dtype=float32)


array([[ 812.99182129],
       [ 812.99694824],
       [ 812.94592285],
       ..., 
       [ 812.99731445],
       [ 812.99493408],
       [ 812.99603271]], dtype=float32)

In [105]:
def reward_prediction(train):
    loss_RNN, predicted_reward = train_lstm_model()
    print predicted_reward
    plt.plot(loss_RNN)
    plt.xlabel('Step number')
    plt.ylabel('Prediction Loss')

In [ ]:
reward_prediction(train)

## Learning the rewards:
Now, the LSTM_RNN output (from validation phase) is considered as input for DQN model, to be able to select the action that has the maximum longtime reward for the customer (highest CLV) .. 

https://arxiv.org/pdf/1602.01580.pdf

## Multilayer Deep Neural Network (DQN):

https://github.com/MorvanZhou/Reinforcement-learning-with-tensorflow/blob/master/contents/5_Deep_Q_Network/RL_brain.py

In [ ]:
## call el fun that predicts next_states and then combine them with the tuple 

In [2]:
def DQN_train(x_inputs, y_outputs):

#     print np.shape(y_outputs)
    
    # Placeholder
    X = tf.placeholder(dtype=tf.float32, shape=[None, np.shape(x_inputs)[1] ])
    Y = tf.placeholder(dtype=tf.float32, shape=[None])


    # Model architecture parameters
    n_dim = np.shape(x_inputs)[1] 
    n_neurons_1 = 40
    n_neurons_2 = 15
    n_target = 1 #np.shape(x_inputs)[0]
    
    batch_size = 500 
    epochs = 100
    
    predicted_output = []
    
        # Initializers
    sigma = 1
    
    #First Q Network
    w1 = tf.Variable(tf.random_uniform([n_dim, n_neurons_1], 0, 0.1))
    bias1 = tf.Variable(tf.random_uniform([n_neurons_1], 0, 0.1))
    
    w2 = tf.Variable(tf.random_uniform([n_neurons_1, n_neurons_2], 0, 0.1))
    bias2 = tf.Variable(tf.random_uniform([n_neurons_2], 0, 0.1))
    
    w3 = tf.Variable(tf.random_uniform([n_neurons_2, n_target], 0, 0.1))
    bias3 = tf.Variable(tf.random_uniform([n_target], 0, 0.1))
    
    
    hidden_1 = tf.nn.relu(tf.matmul(X, w1) + bias1)
    hidden_2 = tf.nn.relu(tf.matmul(hidden_1, w2) + bias2)
    y_ = tf.matmul(hidden_2, w3) + bias3
    
    # initialize variables
    init_op = tf.global_variables_initializer()
    
    # Cost function
    mse = tf.reduce_mean(tf.squared_difference(y_, Y))

    # Optimizer
    opt = tf.train.RMSPropOptimizer(0.0001, 0.99).minimize(mse)

    with tf.Session() as sess:
        sess.run(init_op)
        num_itr = int(np.shape(y_outputs)[0] / batch_size)
        
#         for epoch in range(epochs):
#             for i in range(num_itr):
        predicted_output = sess.run(y_, feed_dict={X: x_inputs, Y: y_outputs})
#     print predicted_output
#         print np.shape(predicted_output)       
#         _, c = sess.run([opt, mse], feed_dict={X: x_inputs, Y: y_outputs})
#         print predicted_output
    return predicted_output.astype(int) #[:, 1]

In [3]:
def Q_learning():
    
    avg_Q = []  
    gamma = 0.9
#     actions = actions_prep(df)

# nepisode > https://stats.stackexchange.com/questions/250943/what-is-the-difference-between-episode-and-epoch-in-deep-q-learning
# means one complete path from state, action, next_s, next_a, reward upuntil terminal state

    nepisod = 10 #2 #np.shape(actions)[1]  ##22+ 
    explore = 0.4
    rewards_, tuplesMx0, tuplesMx1, tuplesMx2, tuplesMx3, tuplesMx4, tuplesMx5, tuplesMx6, tuplesMx7, tuplesMx8, tuplesMx9, tuplesMx10, tuplesMx11 = tuple_()  # nepisod ==> 46, tuplesMx[:,-1] ==> 499   
    
    num_rows = np.shape(tuplesMx0)[0] #-1
    
    Q_predicted = np.zeros([num_rows, 12]) #num_rows, num_columns
    
    best_action = np.zeros([num_rows, 1])
#     Q_optimal = np.zeros([nepisod, num_rows])
    
    for i in xrange(nepisod):
        Q_optimal = rewards_.astype(np.float32) + gamma*Q_predicted.max(axis=1) #np.max(Q_predicted) # returns max value per row !
        print Q_optimal
        Q_predicted0 = DQN_train(tuplesMx0, Q_optimal)
        Q_predicted1 = DQN_train(tuplesMx1, Q_optimal)
        Q_predicted2 = DQN_train(tuplesMx2, Q_optimal)
        Q_predicted3 = DQN_train(tuplesMx3, Q_optimal)
        Q_predicted4 = DQN_train(tuplesMx4, Q_optimal)
        Q_predicted5 = DQN_train(tuplesMx5, Q_optimal)
        Q_predicted6 = DQN_train(tuplesMx6, Q_optimal)
        Q_predicted7 = DQN_train(tuplesMx7, Q_optimal)
        Q_predicted8 = DQN_train(tuplesMx8, Q_optimal)
        Q_predicted9 = DQN_train(tuplesMx9, Q_optimal)
        Q_predicted10 = DQN_train(tuplesMx10, Q_optimal)
        Q_predicted11 = DQN_train(tuplesMx11, Q_optimal)
        Q_predicted = np.column_stack((Q_predicted0, Q_predicted1, Q_predicted2, Q_predicted3,
                                          Q_predicted4, Q_predicted5, Q_predicted6, Q_predicted7,
                                          Q_predicted8, Q_predicted9, Q_predicted10, Q_predicted11))
    
#     for i in Q_optimal:
#         if i == 0.0:
#             i = rewards_.astype(np.float32)
            
    best_action = np.argmax(Q_predicted, axis=1) 
        
    return best_action, Q_optimal

In [ ]:
best_action, Q_optimal = Q_learning()
# best_action, Q_optimal